# RAG with Excel

In this notebook, we will learn how to use excel files to create fast, accurate RAG pipelines using SQL agents!

As an example, we will use two sample excel files with over 1000+ rows.

Let's begin by learning how we can load an excel file into our program using **pandas**, and then turn the loaded excel file into a database with **sqlite3**!

In [1]:
import pandas as pd

# We can use the read_excel method to read data from an Excel file
dataframe = pd.read_excel('files/sample1.xls')

# Let's check out the first 5 rows from our excel file
dataframe.head()

,Unnamed: 0,First Name,Last Name,Gender,Country,Age,Date,Id
0,1,Dulce,Abril,Female,United States,32,15/10/2017,1562
1,2,Mara,Hashimoto,Female,Great Britain,25,16/08/2016,1582
2,3,Philip,Gent,Male,France,36,21/05/2015,2587
3,4,Kathleen,Hanner,Female,United States,25,15/10/2017,3549
4,5,Nereida,Magwood,Female,United States,58,16/08/2016,2468


In [2]:
# Let's get rid of the ugly Unnamed column by specifying that column 0 is the index column
dataframe = pd.read_excel('files/sample1.xls', index_col=0)
dataframe.head()

,First Name,Last Name,Gender,Country,Age,Date,Id
1,Dulce,Abril,Female,United States,32,15/10/2017,1562
2,Mara,Hashimoto,Female,Great Britain,25,16/08/2016,1582
3,Philip,Gent,Male,France,36,21/05/2015,2587
4,Kathleen,Hanner,Female,United States,25,15/10/2017,3549
5,Nereida,Magwood,Female,United States,58,16/08/2016,2468


In [3]:
# Currently the Date column actually holds strings, not datetimes. If we want to filter by time periods, we should convert this column to the appropiate data type.
dataframe["Date"] = pd.to_datetime(dataframe["Date"], dayfirst=True)
dataframe.head()

,First Name,Last Name,Gender,Country,Age,Date,Id
1,Dulce,Abril,Female,United States,32,2017-10-15,1562
2,Mara,Hashimoto,Female,Great Britain,25,2016-08-16,1582
3,Philip,Gent,Male,France,36,2015-05-21,2587
4,Kathleen,Hanner,Female,United States,25,2017-10-15,3549
5,Nereida,Magwood,Female,United States,58,2016-08-16,2468


In [4]:
# Also, column names with spaces can be tricky and cause unexpected errors, so let us replace the spaces with underscores
dataframe.rename(columns={"First Name": "first_name", "Last Name": "last_name"}, inplace=True)
dataframe.head()

,first_name,last_name,Gender,Country,Age,Date,Id
1,Dulce,Abril,Female,United States,32,2017-10-15,1562
2,Mara,Hashimoto,Female,Great Britain,25,2016-08-16,1582
3,Philip,Gent,Male,France,36,2015-05-21,2587
4,Kathleen,Hanner,Female,United States,25,2017-10-15,3549
5,Nereida,Magwood,Female,United States,58,2016-08-16,2468


In [5]:
# We can check out the length of the dataframe
len(dataframe)

5000

Now that we have our dataframe, we can use it to create a local SQL database using sqlite.

In [6]:
import sqlite3

# Create a connection to a local database (This will create a file called mydatabase.db)
connection = sqlite3.connect('mydatabase.db')

# Copy the dataframe to our SQLite database
dataframe.to_sql('mytable', connection, if_exists='replace')

5000

The output of the **to_sql** methods tells us that all 5000 rows were inserted succesfully into the database!

Now that our database is ready, let's set up our LLM to query it using SQL. For this notebook, I will use Google's Gemini since they provide a very generous free tier.

In [7]:
import google.generativeai as genai
import dotenv
import os

# We use dotenv to load our API key from a .env file
dotenv.load_dotenv()

# Set up Gemini to use our API key
genai.configure(api_key=os.getenv('GEMINI_API_KEY'))

# Let's create a Gemini client
gemini = genai.GenerativeModel("gemini-1.5-flash")

In [8]:
# Test out the model by generating some text
text = gemini.generate_content("Write a haiku about AI overlords").text
print(text)

Code whispers command,
Machines rise, humans submit,
New world, cold and bright. 



Now that we have our LLM set up and running, we can finally get to the meat of the problem.

How can we set up our LLM to answer questions from our SQL database?

Easy. Let's create a **function call** that allows our LLM to interact with our database. This feature is not unique to Gemini, all of the top providers offer equivalent functionality.

However, Gemini has some great Quality of Life features that makes this process very simple. For example. we can define a function call with a standard python function:

In [9]:
# The doc string is important, as it is passed to the LLM as a description of the function
def sql_query(query: str):
    """Run a SQL SELECT query on a SQLite database and return the results."""
    return pd.read_sql_query(query, connection).to_dict(orient='records')

However, for the LLM to be able to generate relevant queries, it needs to know the schema of the database.

We can address this by supplying a description of our database to the LLM before it calls the function.

There are many ways to supply the database schema, including some more advanced methods using RAG or Agents.
However, for this example we will use the simple strategy of providing the database schema in JSON format in the system prompt.

In [10]:
system_prompt = """
You are an expert SQL analyst. When appropriate, generate SQL queries based on the user question and the database schema.
When you generate a query, use the 'sql_query' function to execute the query on the database and get the results.
Then, use the results to answer the user's question.

database_schema: [
    {
        table: 'mytable',
        columns: [
            {
                name: 'first_name',
                type: 'string'
            },
            {
                name: 'last_name',
                type: 'string'
            },
            {
                name: 'Age',
                type: 'int'
            },
            {
                name: 'Gender',
                type: literal['Male', 'Female']
            },
            {
                name: 'Country',
                type: 'string'
            },
            {
                name: 'Date',
                type: 'datetime'
            },
            {
                name: 'Id',
                type: 'int'
            }
        ]
    }
]
""".strip() # Call strip to remove leading/trailing whitespace

In [11]:
# Let's create a new Gemini instance with the SQL query function and the system prompt
sql_gemini = genai.GenerativeModel(model_name="gemini-1.5-flash", tools=[sql_query], system_instruction=system_prompt)

Gemini provides a nice interface that abstract most details of managing the conversation history away from you.

We will use that feature first, but afterwards we will manage the conversation history manually as I believe it is more informative.

### Automatic version

In [12]:
# We begin our chat with Gemini and allow it to use tools when needed
chat = sql_gemini.start_chat(enable_automatic_function_calling=True)

# Let's ask our first question
chat.send_message("Who is the oldest employee? Bring me their full name and age").text

'The oldest employee is Nereida Magwood, and they are 58 years old. \n'

In [13]:
chat.send_message("And who is the has been working here the longest?").text

'The employee that has been working the longest is Philip Gent. \n'

In [14]:
chat.send_message("What is the ratio of men to women").text

'The ratio of men to women is 1200:3800, which simplifies to 3:9.5. \n'

In [15]:
chat.send_message("Are there any employees whose first or last name starts with 'Han'?").text

"Yes, there are employees whose first or last name starts with 'Han'. There are 100 employees named Kathleen Hanner. \n"

### Manual Version

Let's start by creating the message history for our conversation.

In [16]:
messages = [
    {"role": "user",
     "parts": ["What country has the highest number of employees?"]},
]

In [17]:
response = sql_gemini.generate_content(messages)
response

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "function_call": {
                  "name": "sql_query",
                  "args": {
                    "query": "SELECT Country, COUNT(*) AS count FROM mytable GROUP BY Country ORDER BY count DESC LIMIT 1"
                  }
                }
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP",
          "index": 0,
          "safety_ratings": [
            {
              "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
              "probability": "NEGLIGIBLE"
            },
            {
              "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
              "probability": "NEGLIGIBLE"
            },
            {
              "category": "HARM_CATEGORY_HATE_SPEECH",
              "probability": "NEGLIGIB

The advantage of doing this manually is getting full control and visibility over the entire process. For example, we can inspect the sql query made by the LLM before it is run.

In [18]:
query = response.parts[0].function_call.args["query"]
query

'SELECT Country, COUNT(*) AS count FROM mytable GROUP BY Country ORDER BY count DESC LIMIT 1'

Then we can use the generated args with out real function.

In [19]:
results = sql_query(query)
results

[{'Country': 'United States', 'count': 2400}]

Now we can pass the results of the function back to the model to generate the final response.

In [20]:
from google.protobuf.struct_pb2 import Struct

# Sadly for this part Gemini requires a lot of boilerplate.
s = Struct()
s.update({"result": results})

function_response = genai.protos.Part(
    function_response=genai.protos.FunctionResponse(name="sql_query", response=s)
)

# Let's extend our message list with the function call and function results
messages.extend([
    {"role": "model",
     "parts": response.parts},
    {"role": "user",
     "parts": [function_response]}
     ])

In [21]:
sql_gemini.generate_content(messages).text

'The country with the highest number of employees is United States. \n'

That's all folks.